In [1]:
import pandas as pd
import re

In [2]:
# --- Day 8: Handheld Halting ---
# Your flight to the major airline hub reaches cruising altitude without incident. While you consider checking the in-flight menu for one of those drinks that come with a little umbrella, you are interrupted by the kid sitting next to you.

# Their handheld game console won't turn on! They ask if you can take a look.

# You narrow the problem down to a strange infinite loop in the boot code (your puzzle input) of the device. You should be able to fix it, but first you need to be able to run the code in isolation.

# The boot code is represented as a text file with one instruction per line of text. Each instruction consists of an operation (acc, jmp, or nop) and an argument (a signed number like +4 or -20).

# acc increases or decreases a single global value called the accumulator by the value given in the argument. For example, acc +7 would increase the accumulator by 7. The accumulator starts at 0. After an acc instruction, the instruction immediately below it is executed next.
# jmp jumps to a new instruction relative to itself. The next instruction to execute is found using the argument as an offset from the jmp instruction; for example, jmp +2 would skip the next instruction, jmp +1 would continue to the instruction immediately below it, and jmp -20 would cause the instruction 20 lines above to be executed next.
# nop stands for No OPeration - it does nothing. The instruction immediately below it is executed next.
# For example, consider the following program:

# nop +0
# acc +1
# jmp +4
# acc +3
# jmp -3
# acc -99
# acc +1
# jmp -4
# acc +6
# These instructions are visited in this order:

# nop +0  | 1
# acc +1  | 2, 8(!)
# jmp +4  | 3
# acc +3  | 6
# jmp -3  | 7
# acc -99 |
# acc +1  | 4
# jmp -4  | 5
# acc +6  |
# First, the nop +0 does nothing. Then, the accumulator is increased from 0 to 1 (acc +1) and jmp +4 sets the next instruction to the other acc +1 near the bottom. After it increases the accumulator from 1 to 2, jmp -4 executes, setting the next instruction to the only acc +3. It sets the accumulator to 5, and jmp -3 causes the program to continue back at the first acc +1.

# This is an infinite loop: with this sequence of jumps, the program will run forever. The moment the program tries to run any instruction a second time, you know it will never terminate.

# Immediately before the program would run an instruction a second time, the value in the accumulator is 5.

# Run your copy of the boot code. Immediately before any instruction is executed a second time, what value is in the accumulator?

In [3]:
# get dataframe from excel
df = pd.read_excel('code.xlsx')
df.head()

,code
0,nop +283
1,acc +26
2,acc +37
3,acc +6
4,jmp +109


In [4]:
code_list = list(df.code)
len(code_list)

631

In [5]:
df.code.str.contains('nop').sum()

66

In [6]:
df.code.str.contains('jmp').sum()

228

In [14]:
# get code_list from pandas dataframe
code_list = list(df.code)

# empty list for index numbers
previous_index_numbers = []

# set index and accumulator to zero
index = 0
acc = 0

# begin loop
running_code = True
while running_code:
    
    # if index is in list of previous indexes end loop, otherwise process current code
    if index in previous_index_numbers:
        
        break
        
    else:

        # add index to previous_index_numbers
        previous_index_numbers.append(index)
        
        # get current code from list
        code = code_list[index]

        # get command, number, and sign for current code
        mod = re.findall(r'(\w*)',code)
        
        # remove '' from mod
        while("" in mod):
            mod.remove('')

        
        command = mod[0]
        amount = int(mod[1])
        sign = re.findall(r'\s(\W)',code)[0]
        
        # if command is 'nop' raise 'index' by 1
        if command == 'nop':

            index += 1

        # if command is 'acc' raise or lower the 'acc' by amount based on the code's sign
        # raise index by 1
        if command == 'acc':
            
            index += 1
            
            if sign == '+':
                
                acc += amount
                
            if sign == '-':
                
                acc -= amount
            
        # if command is 'jmp' raise or lower index by amount based on the code's sign
        if command == 'jmp':
            
            if sign == '+':
                
                index += amount
                
            if sign == '-':
                
                index -= amount

print(acc)

1179


In [ ]:
# --- Part Two ---
# After some careful analysis, you believe that exactly one instruction is corrupted.

# Somewhere in the program, either a jmp is supposed to be a nop, or a nop is supposed to be a jmp. (No acc instructions were harmed in the corruption of this boot code.)

# The program is supposed to terminate by attempting to execute an instruction immediately after the last instruction in the file. By changing exactly one jmp or nop, you can repair the boot code and make it terminate correctly.

# For example, consider the same program from above:

# nop +0
# acc +1
# jmp +4
# acc +3
# jmp -3
# acc -99
# acc +1
# jmp -4
# acc +6
# If you change the first instruction from nop +0 to jmp +0, it would create a single-instruction infinite loop, never leaving that instruction. If you change almost any of the jmp instructions, the program will still eventually find another jmp instruction and loop forever.

# However, if you change the second-to-last instruction (from jmp -4 to nop -4), the program terminates! The instructions are visited in this order:

# nop +0  | 1
# acc +1  | 2
# jmp +4  | 3
# acc +3  |
# jmp -3  |
# acc -99 |
# acc +1  | 4
# nop -4  | 5
# acc +6  | 6
# After the last instruction (acc +6), the program terminates by attempting to run the instruction below the last instruction in the file. With this change, after the program terminates, the accumulator contains the value 8 (acc +1, acc +1, acc +6).

# Fix the program so that it terminates normally by changing exactly one jmp (to nop) or nop (to jmp). What is the value of the accumulator after the program terminates?

In [21]:
code_list = list(df.code)
len(code_list)

631

In [34]:
def terminated(replacement_number,replaced_command,replacement_command):    
    
    # get code_list from pandas dataframe
    code_list = list(df.code)

    # empty list for index numbers
    previous_index_numbers = []

    # set index and accumulator to zero
    index = 0
    acc = 0
    
    # set count to zero
    count = 0
    
    # begin loop
    running_code = True
    while running_code:

        # if index is in list of previous indexes end loop and print did not terminate
        if index in previous_index_numbers:

            print('Did not terminate!!!')
            return False
        
        # if the last number added to previous index numbers is equal to the length of code_list 
        # end loop and Terminated and print 'acc'
        elif index == len(code_list):

            print('Terminated:', acc, index)
            return True

        else:

            # add index to previous_index_numbers
            previous_index_numbers.append(index)

            # get current code from list
            code = code_list[index]

            # get command, number, and sign for current code
            mod = re.findall(r'(\w*)',code)

            # remove '' from mod
            while("" in mod):
                mod.remove('')


            command = mod[0]
            amount = int(mod[1])
            sign = re.findall(r'\s(\W)',code)[0]
            
            # if command equals replaced command add one to the count
            if command == replaced_command:
                
                count += 1
            
            # if count equals replacement_number replace the command
            if count == replacement_number:
                
                command = replacement_command 
                print('change')

            # if command is 'nop' raise 'index' by 1
            if command == 'nop':

                index += 1

            # if command is 'acc' raise or lower the 'acc' by amount based on the code's sign
            # raise index by 1
            if command == 'acc':

                index += 1

                if sign == '+':

                    acc += amount

                if sign == '-':

                    acc -= amount

            # if command is 'jmp' raise or lower index by amount based on the code's sign
            if command == 'jmp':

                if sign == '+':

                    index += amount

                if sign == '-':

                    index -= amount

        

In [35]:
# get total number of relacements for nop and jmp
nops_to_replace = df.code.str.contains('nop').sum()
jmps_to_replace = df.code.str.contains('jmp').sum()

# set replacement number to zero
replacement_number = 0

# begin loop
# increase replacement number by one for each itteration 
testing = True
while testing:

    replacement_number += 1
    
    if terminated(replacement_number,'nop','jmp'):
        
        break
        
    if terminated(replacement_number,'jmp','nop'):
        
        break

change
change
Terminated: 342 631
